# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key
!jupyter nbextension enable --py --sys-prefix widgetsnbextension
!jupyter nbextension enable --py --sys-prefix gmaps

Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: OK
Enabling notebook extension jupyter-gmaps/extension...
      - Validating: OK


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
#create a DataFrame from cites.csv
Weather_C_Heatmap = pd.read_csv("../output_data/cities.csv")
Weather_C_Heatmap

,City,Lat,Long,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Rikitea,-23.1203,-134.9692,72.19,83,10,9.46,PF,1666605983
1,Moose Factory,51.2585,-80.6094,50.36,94,51,5.08,CA,1666605983
2,Masindi,1.6744,31.7150,83.05,46,2,5.46,UG,1666605983
3,Vaini,-21.2000,-175.2000,71.76,64,75,12.66,TO,1666605983
4,Albany,42.6001,-73.9662,54.27,79,100,3.18,US,1666605983
...,...,...,...,...,...,...,...,...,...
575,Laas,46.6166,10.7002,56.35,91,100,3.94,IT,1666605983
576,Quzhou,28.9594,118.8686,64.38,43,100,8.32,CN,1666605983
577,Galveston,29.3669,-94.9669,77.18,80,75,16.11,US,1666605983
578,Akner,39.5349,46.3073,59.59,27,0,4.05,AM,1666605983


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
#Configure gmaps
gmaps.configure(api_key=g_key)

In [4]:
#Latitude and Longitude from the Weather_C_Heatmap will act as locations
WC_Locations = Weather_C_Heatmap[['Lat','Long']]
#Humidity from the Weather_C_Heatmap will act as weight
humid = Weather_C_Heatmap['Humidity'].astype(float)

In [6]:
#Plot Heatmap for data in Weather_C_Heatmap dataframe

#Create a google maps figure
WC_fig = gmaps.figure()

#Create Heatmap layer
WC_heat_layer = gmaps.heatmap_layer(WC_Locations, weights=humid, 
                                 dissipating=False, max_intensity=10,
                                 point_radius=1)

#Add and display the layer

WC_fig.add_layer(WC_heat_layer)
WC_fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [7]:
#Create a DataFrame (GoodTemp_df) based on characteristics of a good vacation:
#Temperature between 70 and 80 degrees fahrenheit.
#No cloud cover
#Wind speed under 10 kph
#Use the dropna() function to drop rows with null values.
GoodTemp_df = Weather_C_Heatmap.loc[(Weather_C_Heatmap[['Max Temp'][0]]>=70) &
                                 (Weather_C_Heatmap[['Max Temp'][0]]<=80) &
                                 (Weather_C_Heatmap[['Cloudiness'][0]]==0) &
                                 (Weather_C_Heatmap[['Wind Speed'][0]]<=10)   
                                ].dropna()
GoodTemp_df

,City,Lat,Long,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
66,Cape Town,-33.9258,18.4232,77.27,54,0,10.00,ZA,1666605983
111,Manavgat,36.7867,31.4431,75.40,29,0,5.68,TR,1666605983
168,Cayenne,4.9333,-52.3333,75.24,100,0,1.14,GF,1666605983
236,Trogir,43.5125,16.2517,75.79,56,0,1.14,HR,1666605983
246,Annau,37.8876,58.5160,71.40,37,0,9.48,TM,1666605983
303,Susurluk,39.9136,28.1578,72.09,36,0,7.34,TR,1666605983
382,Hūn,29.1268,15.9477,72.05,39,0,2.89,LY,1666605983
401,Áris,37.1000,22.0000,78.67,53,0,5.75,GR,1666605983
426,Abu Samrah,35.3029,37.1841,76.59,39,0,5.35,SY,1666605983
442,Ubari,26.5921,12.7805,73.63,23,0,6.82,LY,1666605983


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [8]:
#Create a variable to hold the values for the upcoming loop
GoodTemp_df['Hotel Name'] = ""

In [9]:
#Variable to use with google places API
# The Google API Key 'gkey' has already been defined


params = {
    "radius": 5000,
    "types": "lodging",
    "keyword": "Hotel",
    "key": g_key
}

for index, row in GoodTemp_df.iterrows():
    lat = row['Lat']
    lng = row['Long']
    params['location'] = f'{lat},{lng}'
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    hotel_name = requests.get(base_url, params=params).json()
    try:
        GoodTemp_df.loc[index, 'Hotel Name'] = hotel_name["results"][0]["name"]
    except:
        GoodTemp_df.loc[index, 'Hotel Name'] = 'No Hotel Returned'


In [10]:
# DataFrame with Hotel Names returned from the API calls is 'GoodTemps_df'.
# Remove rows from GoodTemps_df where no hotel was returned
#Create a new DataFrame named 'GoodHotel_df' that only contains actual hotels.
GoodHotel_df = GoodTemp_df.loc[GoodTemp_df[['Hotel Name'][0]]!='No Hotel Returned']
GoodHotel_df

,City,Lat,Long,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
66,Cape Town,-33.9258,18.4232,77.27,54,0,10.00,ZA,1666605983,"Radisson Blu Hotel Waterfront, Cape Town"
111,Manavgat,36.7867,31.4431,75.40,29,0,5.68,TR,1666605983,Kleopatra Side Hotel
168,Cayenne,4.9333,-52.3333,75.24,100,0,1.14,GF,1666605983,Grand Hotel Montabo
236,Trogir,43.5125,16.2517,75.79,56,0,1.14,HR,1666605983,Hotel Medena
303,Susurluk,39.9136,28.1578,72.09,36,0,7.34,TR,1666605983,Grand Rimedya Hotel Susurluk
382,Hūn,29.1268,15.9477,72.05,39,0,2.89,LY,1666605983,Al Nakhel Hotel
521,Key West,24.5557,-81.7826,78.80,87,0,6.91,US,1666605983,NYAH Key West
537,Kumluca,36.3703,30.2869,76.15,45,0,6.60,TR,1666605983,Şahin Otel
547,Shahrud,36.4182,54.9763,71.89,28,0,8.59,IR,1666605983,Paramida hotel


In [13]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in GoodHotel_df.iterrows()]
locations = GoodHotel_df[["Lat", "Long"]]
#locations

In [12]:
#Plot marker layer for data in Goodtemp_df DataFrame
WC_marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

#Create Heatmap layer
WC_fig.add_layer(WC_marker_layer)

#Add and display the layer on top of previously created heatmap layer
WC_fig

Figure(layout=FigureLayout(height='420px'))